In [1]:
######importing libraries for data manipulation#######
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import math
from PIL import Image
import requests
from io import BytesIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.linear_model import Perceptron
from spark_sklearn import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
import mord
from time import time
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st

In [2]:
###############Function definitions###########

#########Function definitions for separating related feature########
def change_vals_new_col(s,value,new_cols):
    
    if(value.get(s) != None):
        if((type(value[s]) == float) and np.isnan(value[s])):
            new_cols[s] = np.nan
        else:
            new_cols.get(s).append(value[s])
    else:
        new_cols.get(s).append(np.nan)
    

def generate_new_cols(related):
    
    new_cols = {'also_bought':[], 'also_viewed':[],'bought_together':[],'buy_after_viewing':[]}
    
    for key,value in related.items():
        if((type(value) == float) and np.isnan(value)):
            
            new_cols['also_bought'].append(np.nan)
            new_cols['also_viewed'].append(np.nan)
            new_cols['bought_together'].append(np.nan)
            new_cols['buy_after_viewing'].append(np.nan)
        else:
            change_vals_new_col('also_bought',value,new_cols)
            change_vals_new_col('also_viewed',value,new_cols)
            change_vals_new_col('bought_together',value,new_cols)
            change_vals_new_col('buy_after_viewing',value,new_cols)
            
        
    return new_cols
#####Function definitions for separating related feature ends####

def plot_related_prods(related,which,final_metadata):
    
    if(related == None):
        print('People who'+str(related)+'this product did not buy any other product:')
        return
    else:
        #print(np.array(related) in final_metadata.index)
        tot = 0
        for idx in related:
            if(idx in final_metadata.index):
                tot += 1
        print(tot)
        tot = round(tot/2)
        print('final',tot)
        f, axes = plt.subplots(tot,tot,figsize=(4,4),dpi=300)
        f.suptitle('People also '+str(which))
        for i in range(0,tot):
            for j in range(0,tot):
                curr_asin = related[i+j]
                if((curr_asin in final_metadata.index) == True):
                    curr_url = final_metadata.loc[curr_asin]['imUrl']
                    curr_title = final_metadata.loc[curr_asin]['title']
                    curr_title = curr_title[0:30]
                    response = requests.get(curr_url)
                    img = Image.open(BytesIO(response.content))
                    axes[i,j].imshow(img)
                    axes[i,j].get_xaxis().set_ticks([])
                    axes[i,j].get_yaxis().set_ticks([])
                    plt.axis('off')
                    axes[i,j].set_title(curr_title,size=3)
        plt.show()            

def Show_related_products(meta_data_row,final_metadata):
    
    #print(meta_data_row)
    curr_url = meta_data_row['imUrl']
    #curr_prod_id = meta_data_row['asin']
    title = meta_data_row['title']
    
    print('The current product is:',title)
    response = requests.get(curr_url)
    img = Image.open(BytesIO(response.content))
    plt.imshow(img)
    plt.show()
    
    ####People who bought this product also bought####
    also_bought = meta_data_row['also_bought']
    if((type(also_bought) == float) and np.isnan(also_bought)):
        also_bought = None
    else:
        if(len(also_bought) > 9):
            also_bought = also_bought[0:9]
    plot_related_prods(also_bought,'bought',final_metadata)
    
    ####People who bought this product also viewed####
    also_viewed = meta_data_row['also_viewed']
    if((type(also_viewed) == float) and np.isnan(also_viewed)):
        also_viewed = None
    else:
        if(len(also_viewed) > 9):
            also_viewed = also_viewed[0:9]
    plot_related_prods(also_viewed,'viewed',final_metadata)


def bootStrap(learner,data,Y,size):
    
    train_acc = []
    test_acc = []
    train_f1 = []
    test_f1 = []
    data_merged = pd.concat([data,Y],axis=1)
    for i in range(0,size):
        #print(i)
        data_sampled = data_merged.sample(5000)
        
        X = data_sampled.iloc[:,0:(data_sampled.shape[1]-1)]
        Y = data_sampled.iloc[:,(data_sampled.shape[1]-1):data_sampled.shape[1]]
        
        #print(X.shape)
        #print(Y.shape)
        
        X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
        
        #print(X_train.shape)
        #print(y_train.shape)
        
        learner = learner.fit(X_train,y_train)
        
        predictions_test = learner.predict(X_test)
        predictions_train = learner.predict(X_train)
        
        train_acc.append(accuracy_score(y_train,predictions_train))
        test_acc.append(accuracy_score(y_test,predictions_test))
        train_f1.append(f1_score(y_train,predictions_train))
        test_f1.append(f1_score(y_test,predictions_test))
        
    
    return train_acc,test_acc,train_f1,test_f1

def Hypothesis_test(sampling_dist,s1):
    
    train_acc_samp = sampling_dist[0] 
    test_acc_samp = sampling_dist[1]
    train_f1_samp = sampling_dist[2]
    test_f1_samp = sampling_dist[3]

    train_acc_mean = np.mean(train_acc_samp)
    train_acc_std = np.std(train_acc_samp)
    train_acc_SE = train_acc_std/np.sqrt(100) 
    print('Train accuracy mean of ',s1,' is',train_acc_mean)
    print('Train accuracy standard deviation of ',s1,' is',train_acc_std)
    print('Train accuracy Standard error of ',s1,' is',train_acc_SE)
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

    test_acc_mean = np.mean(test_acc_samp)
    test_acc_std = np.std(test_acc_samp)
    test_acc_SE = test_acc_std/np.sqrt(100)
    print('Test accuracy mean of ',s1,' is: ',test_acc_mean)
    print('Test accuracy standard deviation of ',s1,' is',test_acc_std)
    print('Test accuracy Standard error of ',s1,' is',test_acc_SE)
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

    train_f1_mean = np.mean(train_f1_samp)
    train_f1_std = np.std(train_f1_samp)
    train_f1_SE = train_f1_std/np.sqrt(100)
    print('Train f1 mean of ',s1,' is',train_f1_mean)
    print('Train f1 standard deviation of ',s1,' is',train_f1_std)
    print('Train f1 Standard error of ',s1,' is',train_f1_SE)
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')

    test_f1_mean = np.mean(test_f1_samp)
    test_f1_std = np.std(test_f1_samp)
    test_f1_SE = test_f1_std/np.sqrt(100)
    print('Test f1 mean of ',s1,' is',test_f1_mean)
    print('Test f1 standard deviation of ',s1,' is',test_f1_std)
    print('Test f1 Standard error of ',s1,' is',test_f1_SE)
    print('-----------------------------------------------------------')
    print('-----------------------------------------------------------')
    
    dist = np.random.normal(loc=test_acc_mean,scale=test_acc_SE,size = 10000)
    density_prop = {"color": "green"}
    hist_prop = {"alpha": 0.3, "color": "red"}
    s = '95 % confidence interval of test accuracy of '+s1
    plot_densityCurve(dist,density_prop,hist_prop,100,5000,test_acc_mean,test_acc_SE,accuracy_naive,s)
    
    dist = np.random.normal(loc=test_f1_mean,scale=test_f1_SE,size = 10000)
    density_prop = {"color": "green"}
    hist_prop = {"alpha": 0.3, "color": "red"}
    s = '95 % confidence interval of test f_beta(0.5) score '+s1
    plot_densityCurve(dist,density_prop,hist_prop,100,5000,test_f1_mean,test_f1_SE,fscore_naive,s1)
    
    
    
def f_beta(y_true,y_pred,beta):
    
    precision = precision_score(y_true,y_pred)
    recall = recall_score(y_true,y_pred)

    # TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
    fscore = ((1+beta*beta)*(precision*recall))/((beta*beta*precision)+recall)
    
    return fscore
    

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_test = X_test.astype(int)
    y_test = y_test.astype(int)
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = learner.fit(X_train[0:sample_size],y_train[0:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end-start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[0:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = accuracy_score(y_train[0:sample_size],predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_test,predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()

    results['f_train'] = f_beta(y_train[0:sample_size],predictions_train,0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    results['f_test'] = f_beta(y_test,predictions_test,0.5)
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
    print("Train accuracy is:", results['acc_train'])
    print("Test accuracy is:", results['acc_test'])
    print("Train F-beta(0.5) score is:", results['f_train'])
    print("Test F-beta(0.5) is:", results['f_test'])
    print('_________________________________________')
    print('_________________________________________')
        
    # Return the results
    return results

def Naive_Bayes(X,Y,which,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    pred_train = clf.predict(X_train)
    print('Train accuracy score of ',which,' NB is: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' NB is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' NB is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' NB is: ',f1_score(y_test, pred_test,average=avg))
    
def SVM(X,Y,which,avg = 'binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    clf = SVC(kernel='rbf',C=2,gamma=20)
    clf.fit(X_train, y_train)
    pred_train = clf.predict(X_train)
    print('Train accuracy score of ',which,' SVM is: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' SVM is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' SVM is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' SVM is: ',f1_score(y_test, pred_test,average=avg))
    
    
def Perceptron_classifier(X,Y,which,weights=None,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    clf = Perceptron(max_iter=200,class_weight=weights)
    clf.fit(X_train,y_train)
    pred_train = clf.predict(X_train)
    
    print('Train accuracy score of ',which,' Perceptron: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' Perceptron is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' Perceptron is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' Perceptron is: ',f1_score(y_test, pred_test,average=avg))
    
def Random_Forest(X,Y,which,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    clf = RandomForestClassifier(n_estimators=200, max_depth=2,random_state=0)
    clf.fit(X_train,y_train)
    pred_train = clf.predict(X_train)
    
    print('Train accuracy score of ',which,' RandomForest: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' RandomForest is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' RandomForest is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' RandomForest is: ',f1_score(y_test, pred_test,average=avg))
    
def Ada_Boost(X,Y,which,avg='binary'):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    clf = AdaBoostClassifier(n_estimators=200)
    clf.fit(X_train,y_train)
    pred_train = clf.predict(X_train)
    
    print('Train accuracy score of ',which,' RandomForest: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 ',which,' RandomForest is: ',f1_score(y_train, pred_train,average=avg))
    pred_test = clf.predict(X_test)
    print('Test accuracy score of ',which,' RandomForest is: ',accuracy_score(y_test, pred_test, normalize=True))
    print('Test F-1 ',which,' RandomForest is: ',f1_score(y_test, pred_test,average=avg))
    
def Ordinal_Logistic_regression(X,Y):
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
    
    X_train = X_train.astype(int)
    y_train = y_train.astype(int)
    X_test = X_test.astype(int)
    y_test = y_test.astype(int)
    
    clf = mord.LogisticIT()
    clf.fit(X_train,y_train)
    
    pred_train = clf.predict(X_train)
    #pred_train = np.round(pred_train,decimals=0)
    #pred_train.astype(int)
    print(pred_train[0:20])
    print(Y[0:20])
    print('Train accuracy score of multiclass ordinl Ridge regression is: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Train F-1 of multiclass ordinl Ridge regression is: ',f1_score(y_train, pred_train,average='weighted'))
    
    pred_test = clf.predict(X_test)
    #pred_test = np.round(pred_test,decimals=0)
    #pred_test.astype(int)
    print('Test accuracy score of multiclass ordinl Ridge regression is: ',accuracy_score(y_train, pred_train, normalize=True))
    print('Test F-1 of multiclass ordinl Ridge regression is: ',f1_score(y_train, pred_train,average='weighted'))

#######Reciever Operating Characteristics definitions###########

    
def ROC_AUC(classifier,X,y,which=None,c=None):
  
    X = np.array(X)
    y = np.array(y)
    y = np.reshape(y,(y.shape[0],1))
    title = 'ROC for Binary labels'
    
    if(which == 'multi'):
        y = label_binarize(y, classes=[1,2,3,4,5])
        title = 'ROC for multi-class labels'
        #f, axes = plt.subplots(3,2,figsize=(8,8),dpi=300)
        #f.suptitle('Distribution of drawing cards simulations')
        #m = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1)]
    else:
        title = 'ROC for Binary labels'
        #f, axes = plt.subplots(1,2,figsize=(8,8),dpi=300)
        #f.suptitle('Distribution of drawing cards simulations')
        #m = [(0,0),(0,1)]
        
        
    print(y.shape)
    random_state = np.random.RandomState(0)
    cv = StratifiedKFold(n_splits=6)
    
    mean_fpr = np.linspace(0, 1, 100)
    
    for t in range(0,y.shape[1]):
        i = 0
        tprs = []
        aucs = []
        #a = m[t][0]
        #b = m[t][1]
        #print(a)
        #print(b)
        for train, test in cv.split(X, y[:,t]):
            #print(y[train].shape)
            
            if(c == 'p'):
                probas_ = classifier.fit(X[train], y[train,t]).score(X[test],y[test,t])
            else:
                probas_ = classifier.fit(X[train], y[train,t]).predict_proba(X[test])
            # Compute ROC curve and area the curve
            
            fpr, tpr, thresholds = roc_curve(y[test,t], probas_[:, 1])
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))        
            i = i + 1
   
        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=2, alpha=.8)
               
        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')
               
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        if(which == 'multi'):
            plt.title(title+' for rating = '+str(t+1))
        else:
            plt.title(title)
        
        plt.legend(loc="lower right")
        plt.show()

def evaluate(results, accuracy, f1):
    """
    Visualization code to display results of various learners.
    
    inputs:
      - learners: a list of supervised learners
      - stats: a list of dictionaries of the statistic results from 'train_predict()'
      - accuracy: The score for the naive predictor
      - f1: The score for the naive predictor
    """
  
    # Create figure
    fig, ax = plt.subplots(2, 3, figsize = (30,20))

    # Constants
    bar_width = 0.15
    colors = ['#A00000','#00A0A0','#00A000','orange','purple']
    
    # Super loop to plot four panels of data
    for k, learner in enumerate(results.keys()):
        for j, metric in enumerate(['train_time', 'acc_train', 'f_train', 'pred_time', 'acc_test', 'f_test']):
            for i in np.arange(3):
                # Creative plot code
                ax[j//3, j%3].bar(i+k*bar_width, results[learner][i][metric], width = bar_width, color = colors[k])
                ax[j//3, j%3].set_xticks([0.45, 1.45, 2.45])
                ax[j//3, j%3].set_xticklabels(["1%", "10%", "100%"])
                ax[j//3, j%3].set_xlabel("Training Set Size",fontsize = 26)
                ax[j//3, j%3].set_xlim((-0.1, 3.0))
    
    # Add unique y-labels
    ax[0, 0].set_ylabel("Time (in seconds)",fontsize = 26)
    ax[0, 1].set_ylabel("Accuracy Score",fontsize = 26)
    ax[0, 2].set_ylabel("F-beta(0.5)",fontsize = 26)
    ax[1, 0].set_ylabel("Time (in seconds)",fontsize = 26)
    ax[1, 1].set_ylabel("Accuracy Score",fontsize = 26)
    ax[1, 2].set_ylabel("F-beta(0.5)",fontsize = 26)
    
    # Add titles
    ax[0, 0].set_title("Model Training",fontsize = 26)
    ax[0, 1].set_title("Accuracy Score on Training Subset",fontsize = 26)
    ax[0, 2].set_title("F-beta(0.5) on Training Subset",fontsize = 26)
    ax[1, 0].set_title("Model Predicting",fontsize = 26)
    ax[1, 1].set_title("Accuracy Score on Testing Set",fontsize = 26)
    ax[1, 2].set_title("F-beta(0.5) on Testing Set",fontsize = 26)
    
    # Add horizontal lines for naive predictors
    ax[0, 1].axhline(y = accuracy, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[1, 1].axhline(y = accuracy, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[0, 2].axhline(y = f1, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    ax[1, 2].axhline(y = f1, xmin = -0.1, xmax = 3.0, linewidth = 1, color = 'k', linestyle = 'dashed')
    
    # Set y-limits for score panels
    ax[0, 1].set_ylim((0, 1))
    ax[0, 2].set_ylim((0, 1))
    ax[1, 1].set_ylim((0, 1))
    ax[1, 2].set_ylim((0, 1))

    # Create patches for the legend
    patches = []
    for i, learner in enumerate(results.keys()):
        patches.append(mpatches.Patch(color = colors[i], label = learner))
    plt.legend(handles = patches,bbox_to_anchor=(0.5, 1,0.5,0.5), loc='upper center',ncol = 3, fontsize = 26)
    
    # Aesthetics
    #plt.figlegend( 'Ruchin', 'Patel', loc = 'lower center', ncol=5, labelspacing=0. )
    plt.suptitle("Performance Metrics for 5 Supervised Learning Models", fontsize = 26, y = 1.10)
    plt.tight_layout()
    plt.show()
        
def plot_densityCurve(*args):
    plt.figure(figsize=(9,4), dpi=300)
    sns.distplot(args[0],kde_kws=args[1])
    plt.axvline(args[5], color='yellow', linestyle='-.', linewidth=1,label='sample mean')
    plt.axvline(args[5]-args[6], color='black', linestyle=':', linewidth=1,label='1 standard dev')
    plt.axvline(args[5]+args[6], color='black', linestyle=':', linewidth=1)
    plt.axvline(args[7], color='purple', linestyle='-.', linewidth=2,label='True mean')
    plt.axvline(args[5]-(1.96*args[6]),color='black',linewidth=2,label='95% confidence line')
    plt.axvline(args[5]+(1.96*args[6]),color='black',linewidth=2)
    #plt.xlim(0.72,0.85)
    plt.legend()
    #plt.title("The sampling distribution with "+str(args[3])+" samples of size n="+str(args[4]))
    plt.title(args[8])
    plt.show()  
    
def plot_norm(sample_mean,SE,*args):
    plt.figure(figsize=(9,4), dpi=300)
    x_values = np.random.normal(loc = sample_mean,scale=SE,size=args[0])
    x_values = np.sort(x_values)
    y_values = st.norm.pdf(x_values,loc = sample_mean,scale=SE)
    plt.plot(x_values,y_values,linewidth=2,color='green')
    plt.axvline(sample_mean, color='yellow', linestyle='-.', linewidth=2,label='sample mean')
    plt.axvline(sample_mean-SE, color='black', linestyle=':', linewidth=1,label='1 standard dev')
    plt.axvline(sample_mean+SE, color='black', linestyle=':', linewidth=1)
    plt.axvline(args[1], color='purple', linestyle='-', linewidth=1,label='95% confidence line')
    plt.axvline(args[2], color='purple', linestyle='-', linewidth=1)
    x_95 = x_values[np.logical_and(x_values>=args[1],x_values<=args[2])]
    y_95 = y_values[np.logical_and(x_values>=args[1],x_values<=args[2])]
    plt.fill_between(x_95,0,y_95,color='red',alpha=0.4)
    #plt.ylim(0,10)
    plt.legend()
    plt.show()  
        